In [ ]:
import pandas as pd
from time import time
offers=pd.read_csv('offers.csv')
out=offers[(offers.category>9000)  & (offers.category>9000)]

In [ ]:
trainHistory=pd.read_csv('trainHistory.csv')
print trainHistory[trainHistory.id == 242209515]
print len(trainHistory)

In [ ]:
offer_train=pd.merge(offers,trainHistory, on='offer')
print offer_train.iloc[0]

In [ ]:
from time import time
t0=time()
train_transaction_category=pd.read_csv('transactions_match_category_train.csv')
train_transaction_category.set_index(['category','date'])
t1=time()
print 'total time='+str(t1-t0)
#print transaction_category

In [ ]:
train_transaction_company=pd.read_csv('transactions_match_company_train.csv')
train_transaction_company.set_index(['company','date'])
#print transaction

In [ ]:
train_transaction_brand=pd.read_csv('transactions_match_brand_train.csv')
train_transaction_brand.set_index(['brand','date'])
#print transaction

In [ ]:
train_transaction_all3=pd.read_csv('transactions_reduced_all3_train.csv')
#print transaction

In [ ]:
train_transaction_company_joined=pd.merge(train_transaction_company,trainHistory, on=['id','chain'])
train_transaction_company_joined.set_index(['id', 'chain', 'company', 'date'])

In [ ]:
train_transaction_category_joined=pd.merge(train_transaction_category,trainHistory, on=['id','chain'])
train_transaction_category_joined.set_index(['id', 'chain', 'category', 'date'])

In [ ]:
train_transaction_brand_joined=pd.merge(train_transaction_brand,trainHistory, on=['id','chain'])
train_transaction_brand_joined.set_index(['id', 'chain', 'brand', 'date'])

In [ ]:
train_transaction_all_joined=pd.merge(train_transaction_all3,trainHistory, on=['id','chain'])
train_transaction_all_joined.set_index(['id', 'chain', 'category','date'])

In [ ]:
feature_names = ['S_category_180',
'P_amt_category_180',
'S_category_120',
'P_amt_category_120',
'S_category_90',
'P_amt_category_90',
'S_category_60',
'P_amt_category_60',
'S_category_30',
'P_amt_category_30',
'S_company_180',
'P_amt_company_180',
'S_company_120',
'P_amt_company_120',
'S_company_90',
'P_amt_company_90',
'S_company_60',
'P_amt_company_60',
'S_company_30',
'P_amt_company_30',
'S_brand_180',
'P_amt_brand_180',
'S_brand_120',
'P_amt_brand_120',
'S_brand_90',
'P_amt_brand_90',
'S_brand_60',
'P_amt_brand_60',
'S_brand_30',
'P_amt_brand_30',
'S_all_180',
'P_amt_all_180',
'S_all_120',
'P_amt_all_120',
'S_all_90',
'P_amt_all_90',
'S_all_60',
'P_amt_all_60',
'S_all_30',
'P_amt_all_30',
'S_dept_180',
'P_amt_dept_180',
'S_dept_30',
'P_amt_dept_30',
'NE_category_180',                 
'NE_company_180',
'NE_brand_180',
'NE_category_30',                 
'NE_company_30',
'NE_brand_30',
'D_brand_180',
'D_company_180',
'D_category_180',
'D_brand_30',
'D_company_30',
'D_category_30',
'P_amt_diff_brand_30',
'P_amt_diff_brand_180',    
'P_amt_diff_company_30',
'P_amt_diff_company_180',    
'P_amt_diff_category_30',
'P_amt_diff_category_180',
'category_pop',
'company_pop',
'brand_pop',
'offer_value',
'offer_quantity',
'offer_category',
'offer_brand',
'offer_company']


columns = {}
for i in feature_names:
        column_name=i
        columns[column_name]=[]
print columns


In [ ]:

t0=time()
train_transaction_category_joined_groupby_id = train_transaction_category_joined.groupby('id')
train_transaction_company_joined_groupby_id = train_transaction_company_joined.groupby('id')
train_transaction_brand_joined_groupby_id = train_transaction_brand_joined.groupby('id')
train_transaction_all_joined_groupby_id = train_transaction_all_joined.groupby('id')
transaction_category_groups = train_transaction_category_joined.groupby('category')
transaction_company_groups = train_transaction_company_joined.groupby('company')
transaction_brand_groups = train_transaction_brand_joined.groupby('brand')
t1=time()

In [ ]:
pop_category = {}
pop_company = {}
pop_brand = {}

for name, group in transaction_category_groups:
    pop_category[name] = len(group)
for name, group in transaction_company_groups:
    pop_company[name] = len(group)
for name, group in transaction_brand_groups:
    pop_brand[name] = len(group)


In [ ]:
from datetime import datetime
from datetime import timedelta
import numpy as np
import sys
import copy
import os.path

dept = {}
cnt = 0

f = open('log3.txt', 'w')

def noOfPurchaseWithCategory(transactions,category,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x: (x.category==category) &(start_date<= x.date) & (x.date<=end_date), axis = 1)]    
    purchasequantity = np.asarray(results['purchasequantity'])
    purchaseamount = np.asarray(results['purchaseamount'])
    product_sum = np.dot(purchasequantity, purchaseamount)

    return results.shape[0], product_sum

def noOfPurchaseWithCompany(transactions,company,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x: (x.company==company) &(start_date<= x.date) & (x.date<=end_date), axis = 1)]
    
    purchasequantity = np.asarray(results['purchasequantity'])
    purchaseamount = np.asarray(results['purchaseamount'])
    product_sum = np.dot(purchasequantity, purchaseamount)
    
    return results.shape[0], product_sum

def noOfPurchaseWithBrand(transactions,brand,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x: (x.brand==brand) &(start_date<= x.date) & (x.date<=end_date), axis = 1)]

    purchasequantity = np.asarray(results['purchasequantity'])
    purchaseamount = np.asarray(results['purchaseamount'])
    product_sum = np.dot(purchasequantity, purchaseamount)
    
    return results.shape[0], product_sum

def noOfPurchaseWithDept(transactions,category,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    
    if category not in dept:
        department_df = transactions[transactions.apply(lambda x: x.category==category, axis = 1)]
        dept_id = department_df['dept']
        for i in dept_id.index:
            deptid = dept_id[i]
            break
        
        dept[category] = deptid
    else:
        deptid = dept[category]
        

    results=transactions[transactions.apply(lambda x: (start_date<= x.date) & (x.date<=end_date) & (x.dept==deptid), axis = 1)]
    purchasequantity = np.asarray(results['purchasequantity'])
    purchaseamount = np.asarray(results['purchaseamount'])
    product_sum = np.dot(purchasequantity, purchaseamount)

    return results.shape[0], product_sum

def noOfPurchaseWithAll(transactions,category, brand, company,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x:(x.category==category) & (x.brand==brand)& (x.company==company) & (start_date<= x.date) & (x.date<=end_date), axis = 1)]
    
    purchasequantity = np.asarray(results['purchasequantity'])
    purchaseamount = np.asarray(results['purchaseamount'])
    product_sum = np.dot(purchasequantity, purchaseamount)
    
    return results.shape[0], product_sum

def noOfPurchaseWithCategoryDiff(transactions,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    
    results=transactions[transactions.apply(lambda x:(start_date<= x.date) & (x.date<=end_date), axis = 1)]
    diff_category = results.groupby('category')
    
    purchasequantity = np.asarray(results['purchasequantity'])
    purchaseamount = np.asarray(results['purchaseamount'])
    product_sum = np.dot(purchasequantity, purchaseamount)
    
    return len(diff_category), product_sum

def noOfPurchaseWithCompanyDiff(transactions,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    
    results=transactions[transactions.apply(lambda x:(start_date<= x.date) & (x.date<=end_date), axis = 1)]
    diff_company = results.groupby('company')

    purchasequantity = np.asarray(results['purchasequantity'])
    purchaseamount = np.asarray(results['purchaseamount'])
    product_sum = np.dot(purchasequantity, purchaseamount)
    
    return len(diff_company), product_sum


def noOfPurchaseWithBrandDiff(transactions,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    
    results=transactions[transactions.apply(lambda x:(start_date<= x.date) & (x.date<=end_date), axis = 1)] 
    diff_brand = results.groupby('brand')

    purchasequantity = np.asarray(results['purchasequantity'])
    purchaseamount = np.asarray(results['purchaseamount'])
    product_sum = np.dot(purchasequantity, purchaseamount)
    
    return len(diff_brand), product_sum

def noOfPurchaseWithCategoryNegative(transactions,category,offerDate,period):
    sys.stdout.flush()
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x:(x.category==category) & (start_date<= x.date) & (x.date<=end_date) & (x.purchaseamount<0.0), axis = 1)]

    return results.shape[0]

def noOfPurchaseWithCompanyNegative(transactions,company,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x:(x.company==company) & (start_date<= x.date) & (x.date<=end_date) & (x.purchaseamount<0.0), axis = 1)]

    return results.shape[0]

def noOfPurchaseWithBrandNegative(transactions,brand,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x:(x.brand==brand) & (start_date<= x.date) & (x.date<=end_date)& (x.purchaseamount<0.0), axis = 1)]
    
    return results.shape[0]

def noOfPurchaseWithAllNegative(transactions,category, brand, company,offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x: (x.category==category) & (x.brand==brand)& (x.company==company)  & (start_date<= x.date) & (x.date<=end_date) & (x.purchaseamount<0.0), axis = 1)]

    return results.shape[0]

def findPopularity(transactions, offerDate,period):
    endDate=datetime.strptime(offerDate,'%Y-%m-%d')
    tmDelta=timedelta(days=period)
    startDate=endDate-tmDelta
    start_date = startDate.strftime('%Y-%m-%d')
    end_date = endDate.strftime('%Y-%m-%d')
    results=transactions[transactions.apply(lambda x: (start_date<= x.date) & (x.date<=end_date), axis = 1)]
    return results.shape[0]
    
train_category = offer_train['category']
train_company = offer_train['company']
train_brand = offer_train['brand']
train_id = offer_train['id']
train_date = offer_train['offerdate']
train_offer_value = offer_train['offervalue']
train_offer_quantity = offer_train['quantity']

train_file_name = 'train_features_final.csv'

rows=len(train_date)
start = 0
end = 160057
rows_read=0
exists = os.path.exists(train_file_name)
if exists == True:
    file_dataframe = pd.read_csv(train_file_name)
    rows_read = len(file_dataframe)
        
            
row_cnt = 50

columns_temp = copy.deepcopy(columns)

for i in range(start+rows_read, end):
    t0=time()
    
    try:
        transaction_category = train_transaction_category_joined_groupby_id.get_group(train_id[i])
        S_category_180, P_amt_category_180 = noOfPurchaseWithCategory(transaction_category,train_category[i],train_date[i],180)
        S_category_120, P_amt_category_120 = noOfPurchaseWithCategory(transaction_category,train_category[i],train_date[i],120)
        S_category_90, P_amt_category_90 = noOfPurchaseWithCategory(transaction_category,train_category[i],train_date[i],90)
        S_category_60, P_amt_category_60 = noOfPurchaseWithCategory(transaction_category,train_category[i],train_date[i],60)
        S_category_30, P_amt_category_30 = noOfPurchaseWithCategory(transaction_category,train_category[i],train_date[i],30)
        D_Category_180, P_amt_diff_category_180 = noOfPurchaseWithCategoryDiff(transaction_category,train_date[i],180)
        D_Category_30, P_amt_diff_category_30 = noOfPurchaseWithCategoryDiff(transaction_category,train_date[i],30)
        S_dept_180, P_amt_dept_180 = noOfPurchaseWithDept(transaction_category,train_category[i],train_date[i],180)
        S_dept_30, P_amt_dept_30 = noOfPurchaseWithDept(transaction_category,train_category[i],train_date[i],30)
        NE_category_180 = noOfPurchaseWithCategoryNegative(transaction_category,train_category[i],train_date[i],180)
        NE_category_30 = noOfPurchaseWithCategoryNegative(transaction_category,train_category[i],train_date[i],30)
    except:
        S_category_180 = 0
        P_amt_category_180 = 0
        S_category_120 = 0
        P_amt_category_120 = 0
        S_category_90 = 0
        P_amt_category_90 = 0
        S_category_60 = 0
        P_amt_category_60 = 0
        S_category_30 = 0
        P_amt_category_30 = 0
        D_Category_180 = 0
        D_Category_30 = 0
        S_dept_180 = 0
        P_amt_dept_180 = 0
        S_dept_30 = 0
        P_amt_dept_30 = 0
        NE_category_180 = 0
        NE_category_30 = 0
        P_amt_diff_category_180 = 0
        P_amt_diff_category_30 = 0
        
    columns_temp['S_category_180'].append(S_category_180)
    columns_temp['P_amt_category_180'].append(P_amt_category_180)
    columns_temp['S_category_120'].append(S_category_120)
    columns_temp['P_amt_category_120'].append(P_amt_category_120)
    columns_temp['S_category_90'].append(S_category_90)
    columns_temp['P_amt_category_90'].append(P_amt_category_90)
    columns_temp['S_category_60'].append(S_category_60)
    columns_temp['P_amt_category_60'].append(P_amt_category_60)
    columns_temp['S_category_30'].append(S_category_30)
    columns_temp['P_amt_category_30'].append(P_amt_category_30)
    columns_temp['S_dept_30'].append(S_dept_30)
    columns_temp['P_amt_dept_30'].append(P_amt_dept_30)
    columns_temp['S_dept_180'].append(S_dept_180)
    columns_temp['P_amt_dept_180'].append(P_amt_dept_180)
    columns_temp['D_category_30'].append(D_Category_30)
    columns_temp['D_category_180'].append(D_Category_180)
    columns_temp['P_amt_diff_category_30'].append(P_amt_diff_category_30)
    columns_temp['P_amt_diff_category_180'].append(P_amt_diff_category_180)
    columns_temp['NE_category_30'].append(NE_category_30)
    columns_temp['NE_category_180'].append(NE_category_180)
    
    try:
        transaction_company = train_transaction_company_joined_groupby_id.get_group(train_id[i])
        S_company_180, P_amt_company_180 = noOfPurchaseWithCompany(transaction_company,train_company[i],train_date[i],180)
        S_company_120, P_amt_company_120 = noOfPurchaseWithCompany(transaction_company,train_company[i],train_date[i],120)
        S_company_90, P_amt_company_90 = noOfPurchaseWithCompany(transaction_company,train_company[i],train_date[i],90)
        S_company_60, P_amt_company_60 = noOfPurchaseWithCompany(transaction_company,train_company[i],train_date[i],60)
        S_company_30, P_amt_company_30 = noOfPurchaseWithCompany(transaction_company,train_company[i],train_date[i],30)
        NE_company_180 = noOfPurchaseWithCompanyNegative(transaction_company,train_company[i],train_date[i],180)
        NE_company_30 = noOfPurchaseWithCompanyNegative(transaction_company,train_company[i],train_date[i],30)
        D_company_180, P_amt_diff_company_180 = noOfPurchaseWithCompanyDiff(transaction_company,train_date[i],180)
        D_company_30, P_amt_diff_company_30 = noOfPurchaseWithCompanyDiff(transaction_company,train_date[i],30)
    except:
        S_company_180 = 0
        P_amt_company_180 = 0
        S_company_120 = 0
        P_amt_company_120 = 0
        S_company_90 = 0
        P_amt_company_90 = 0
        S_company_60 = 0
        P_amt_company_60 = 0
        S_company_30 = 0
        P_amt_company_30 = 0
        NE_company_180 = 0
        NE_company_30 = 0
        D_company_180 = 0
        D_company_30 = 0
        P_amt_diff_company_180 = 0
        P_amt_diff_company_30 = 0
        
    columns_temp['S_company_180'].append(S_company_180)
    columns_temp['P_amt_company_180'].append(P_amt_company_180)
    columns_temp['S_company_120'].append(S_company_120)
    columns_temp['P_amt_company_120'].append(P_amt_company_120)
    columns_temp['S_company_90'].append(S_company_90)
    columns_temp['P_amt_company_90'].append(P_amt_company_90)
    columns_temp['S_company_60'].append(S_company_60)
    columns_temp['P_amt_company_60'].append(P_amt_company_60)
    columns_temp['S_company_30'].append(S_company_30)
    columns_temp['P_amt_company_30'].append(P_amt_company_30)
    columns_temp['D_company_30'].append(D_company_30)
    columns_temp['D_company_180'].append(D_company_180)
    columns_temp['P_amt_diff_company_30'].append(P_amt_diff_company_30)
    columns_temp['P_amt_diff_company_180'].append(P_amt_diff_company_180)
    columns_temp['NE_company_30'].append(NE_company_30)
    columns_temp['NE_company_180'].append(NE_company_180)

    try:
        transaction_brand = train_transaction_brand_joined_groupby_id.get_group(train_id[i])
        S_brand_30,  P_amt_brand_30 = noOfPurchaseWithBrand(transaction_brand,train_brand[i],train_date[i],30)
        S_brand_120,  P_amt_brand_120 = noOfPurchaseWithBrand(transaction_brand,train_brand[i],train_date[i],120)
        S_brand_90,  P_amt_brand_90 = noOfPurchaseWithBrand(transaction_brand,train_brand[i],train_date[i],90)
        S_brand_60,  P_amt_brand_60 = noOfPurchaseWithBrand(transaction_brand,train_brand[i],train_date[i],60)
        S_brand_180,  P_amt_brand_180 = noOfPurchaseWithBrand(transaction_brand,train_brand[i],train_date[i],180)
        NE_brand_180 = noOfPurchaseWithBrandNegative(transaction_brand,train_brand[i],train_date[i],180)
        NE_brand_30 = noOfPurchaseWithBrandNegative(transaction_brand,train_brand[i],train_date[i],30)
        D_brand_180, P_amt_diff_brand_180 = noOfPurchaseWithBrandDiff(transaction_brand,train_date[i],180)
        D_brand_30, P_amt_diff_brand_30 = noOfPurchaseWithBrandDiff(transaction_brand,train_date[i],30)
    except:
        S_brand_30 = 0
        P_amt_brand_30 = 0
        S_brand_180 = 0
        P_amt_brand_180 = 0
        S_brand_120 = 0
        P_amt_brand_120 = 0
        S_brand_90 = 0
        P_amt_brand_90 = 0
        S_brand_60 = 0
        P_amt_brand_60 = 0
        NE_brand_180 = 0
        NE_brand_30 = 0
        D_brand_180 = 0
        D_brand_30 = 0
        P_amt_diff_brand_30 = 0
        P_amt_diff_brand_180 = 0
        
    columns_temp['S_brand_180'].append(S_brand_180)
    columns_temp['P_amt_brand_180'].append(P_amt_brand_180)
    columns_temp['S_brand_30'].append(S_brand_30)
    columns_temp['P_amt_brand_30'].append(P_amt_brand_30)
    columns_temp['S_brand_60'].append(S_brand_60)
    columns_temp['P_amt_brand_60'].append(P_amt_brand_60)
    columns_temp['S_brand_90'].append(S_brand_90)
    columns_temp['P_amt_brand_90'].append(P_amt_brand_90)
    columns_temp['S_brand_120'].append(S_brand_120)
    columns_temp['P_amt_brand_120'].append(P_amt_brand_120)
    columns_temp['D_brand_30'].append(D_brand_30)
    columns_temp['D_brand_180'].append(D_brand_180)
    columns_temp['P_amt_diff_brand_30'].append(P_amt_diff_brand_30)
    columns_temp['P_amt_diff_brand_180'].append(P_amt_diff_brand_180)
    columns_temp['NE_brand_30'].append(NE_brand_30)
    columns_temp['NE_brand_180'].append(NE_brand_180)
    
    try:
        transaction_all3 = train_transaction_all_joined_groupby_id.get_group(train_id[i])
        S_all_180, P_amt_all_180 = noOfPurchaseWithAll(transaction_all3, train_category[i], train_brand[i], train_company[i],train_date[i],180)
        S_all_120, P_amt_all_120 = noOfPurchaseWithAll(transaction_all3, train_category[i], train_brand[i], train_company[i],train_date[i],120)
        S_all_90, P_amt_all_90 = noOfPurchaseWithAll(transaction_all3, train_category[i], train_brand[i], train_company[i],train_date[i],90)
        S_all_60, P_amt_all_60 = noOfPurchaseWithAll(transaction_all3, train_category[i], train_brand[i], train_company[i],train_date[i],60)
        S_all_30, P_amt_all_30 = noOfPurchaseWithAll(transaction_all3, train_category[i], train_brand[i], train_company[i],train_date[i],30)
       
    except:
        S_all_180 = 0
        P_amt_all_180 = 0
        S_all_120 = 0
        P_amt_all_120 = 0
        S_all_90 = 0
        P_amt_all_90 = 0
        S_all_60 = 0
        P_amt_all_60 = 0
        S_all_30 = 0
        P_amt_all_30 = 0
        
    columns_temp['S_all_180'].append(S_all_180)
    columns_temp['P_amt_all_180'].append(P_amt_all_180)     
    columns_temp['S_all_120'].append(S_all_120)
    columns_temp['P_amt_all_120'].append(P_amt_all_120)   
    columns_temp['S_all_90'].append(S_all_90)
    columns_temp['P_amt_all_90'].append(P_amt_all_90)   
    columns_temp['S_all_60'].append(S_all_60)
    columns_temp['P_amt_all_60'].append(P_amt_all_60)   
    columns_temp['S_all_30'].append(S_all_30)
    columns_temp['P_amt_all_30'].append(P_amt_all_30)  
    columns_temp['offer_value'].append(train_offer_value[i])
    columns_temp['offer_quantity'].append(train_offer_quantity[i])
    columns_temp['offer_brand'].append(train_brand[i])
    columns_temp['offer_category'].append(train_category[i])
    columns_temp['offer_company'].append(train_company[i])
    

    category_pop = pop_category[train_category[i]]
    company_pop = pop_company[train_company[i]]
    brand_pop = pop_brand[train_brand[i]]
    #category_pop_30 = findPopularity(transaction_category_group, train_date[i],30)
    #category_pop_180 = findPopularity(transaction_category_group, train_date[i],180)
    #company_pop_30 = findPopularity(transaction_company_group, train_date[i],30)
    #company_pop_180 = findPopularity(transaction_company_group, train_date[i],180)
    #brand_pop_30 = findPopularity(transaction_brand_group, train_date[i],30)
    #brand_pop_180 = findPopularity(transaction_brand_group, train_date[i],180)
    #columns_temp['category_pop_30'].append(category_pop_30)
    #columns_temp['category_pop_180'].append(category_pop_180)
    #columns_temp['company_pop_30'].append(company_pop_30)
    #columns_temp['company_pop_180'].append(company_pop_180)
    #columns_temp['brand_pop_30'].append(brand_pop_30)
    #columns_temp['brand_pop_180'].append(brand_pop_180)
    columns_temp['category_pop'].append(category_pop)
    columns_temp['company_pop'].append(company_pop)
    columns_temp['brand_pop'].append(brand_pop)
        
    print 'row_read:'+str(i)
    
    t1=time()
    print 'total time='+str(t1-t0)
    sys.stdout.flush()
    
    if (i == end-1):
        df = pd.DataFrame(columns_temp)
        exists = os.path.exists(train_file_name)
        if exists == True:
            with open(train_file_name, 'a') as f:
                df.to_csv(f, header=False)
        else:
            df.to_csv(train_file_name, header=True)
        columns_temp = copy.deepcopy(columns)
            
#We need to call the above functions with the corresponding transaction value( depending on separate csv files)